<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 15px; height: 80px">

# Project 3

### Regression and Classification with the Ames Housing Data

---

You have just joined a new "full stack" real estate company in Ames, Iowa. The strategy of the firm is two-fold:
- Own the entire process from the purchase of the land all the way to sale of the house, and anything in between.
- Use statistical analysis to optimize investment and maximize return.

The company is still small, and though investment is substantial the short-term goals of the company are more oriented towards purchasing existing houses and flipping them as opposed to constructing entirely new houses. That being said, the company has access to a large construction workforce operating at rock-bottom prices.

This project uses the [Ames housing data recently made available on kaggle](https://www.kaggle.com/c/house-prices-advanced-regression-techniques).

In [318]:
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

sns.set_style('whitegrid')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 1. Estimating the value of homes from fixed characteristics.

---

Your superiors have outlined this year's strategy for the company:
1. Develop an algorithm to reliably estimate the value of residential houses based on *fixed* characteristics.
2. Identify characteristics of houses that the company can cost-effectively change/renovate with their construction team.
3. Evaluate the mean dollar value of different renovations.

Then we can use that to buy houses that are likely to sell for more than the cost of the purchase plus renovations.

Your first job is to tackle #1. You have a dataset of housing sale data with a huge amount of features identifying different aspects of the house. The full description of the data features can be found in a separate file:

    housing.csv
    data_description.txt
    
You need to build a reliable estimator for the price of the house given characteristics of the house that cannot be renovated. Some examples include:
- The neighborhood
- Square feet
- Bedrooms, bathrooms
- Basement and garage space

and many more. 

Some examples of things that **ARE renovate-able:**
- Roof and exterior features
- "Quality" metrics, such as kitchen quality
- "Condition" metrics, such as condition of garage
- Heating and electrical components

and generally anything you deem can be modified without having to undergo major construction on the house.

---

**Your goals:**
1. Perform any cleaning, feature engineering, and EDA you deem necessary.
- Be sure to remove any houses that are not residential from the dataset.
- Identify **fixed** features that can predict price.
- Train a model on pre-2010 data and evaluate its performance on the 2010 houses.
- Characterize your model. How well does it perform? What are the best estimates of price?

> **Note:** The EDA and feature engineering component to this project is not trivial! Be sure to always think critically and creatively. Justify your actions! Use the data description file!

In [319]:
# Load the data
house = pd.read_csv('./housing.csv')

In [320]:
# END GOAL: Develop an algorithm to reliably estimate the value of residential 
# houses based on FIXED CHARACTERISTICS.

# Step 1: Identify and isolate the target and predictor variables
# Predictor variables will be features of the house that are fixed
# Target variable will be the sale price

# Step 2: Split the data into pre-2010 data and 2010 data

# Step 3: Train and fit a suitable model

In [321]:
# Looks like there are many null values present in the dataset, need to examine how many
house.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [322]:
house.shape
# House data has 1460 rows and 81 columns

(1460, 81)

In [323]:
null_df = pd.DataFrame(house.isnull().sum().sort_values(ascending=False))
null_df.head(20)

# Seems that there are many columns that have null values
# Displayed are the columns that have null values
# PoolQC, MiscFeature, Alley, Fence, FireplaceQu, LotFrontage are all pretty bad
# Alot of null values in the columns
# Probably should explore what each feature represents

# PoolQC - Represents pool quality
# MiscFeature - Miscellaneous feature not covered in other categories
# Alley - Type of alley access to property (Gravel/Paved/No alley access)
# Fence - Fence Quality
# FireplaceQu - Quality of fireplace
# LotFrontage - Linear feet of street connected to property

# The number of null values for garage columns are all the same (81)
# This probably because 81 houses within the dataset don't have garage
# GarageCond - Condition of Garage
# GarageType - Type of Garage
# GarageYrBlt - Year garage was built
# GarageFinish - Interior finish of the garage
# GarageQual - Quality of the garage

# BsmtExposure - Walkout/garden level walls
# BsmtFinType2 - Rating of finished basement area (how good quality the basement is)
# BsmtFinType1 - " "
# BsmtCond - General condition of the basement
# BsmtQual - Height of the basement

# MasVnrArea - Masonry Veneer area in square feet
# MasVnrType - Masonry Veneer type
# Electrical - Electrical System

,0
PoolQC,1453
MiscFeature,1406
Alley,1369
Fence,1179
FireplaceQu,690
LotFrontage,259
GarageCond,81
GarageType,81
GarageYrBlt,81
GarageFinish,81


In [324]:
# I'll need to look at what's fixed and what's not fixed first before I decide
# What to do with those columns with nulls
# Fixed Definition: Given characteristics of the house that cannot be renovated
# Examples include:
# The neighborhood, Square feet, Bedrooms, bathrooms, Basement and garage space

# Unfixed features definition: Generally anything that can be modified 
# without having to undergo major construction on the house.
# Examples include:
# Roof and exterior features, "Quality" metrics, such as kitchen quality
# "Condition" metrics, such as condition of garage, Heating and 
# electrical components

In [325]:
"""
List of fixed features (Sorted according to definition to the best of my ability):
Categorical
1. MSSubClass 
2. MSZoning 
3. Neighborhood 
4. Bldgtype 
5. HouseStyle 
6. MasVnrType 
7. Foundation 
8. BsmtQual 

Continuous
1. MasVnrArea 
2. TotalBsmtSF 
3. 1stFlrSF 
4. 2ndFltSF 
5. LowQualFinSF 
6. GrLivArea
7. BsmtFullBath
8. BsmtHalfBath
9. FullBath
10. HalfBath
11. Bedroom
12. Kitchen
13. TotRmsAbvGrd
14. Fireplaces
15. GarageCars
16. GarageArea
17. PoolArea

"""

'\nList of fixed features (Sorted according to definition to the best of my ability):\nCategorical\n1. MSSubClass \n2. MSZoning \n3. Neighborhood \n4. Bldgtype \n5. HouseStyle \n6. MasVnrType \n7. Foundation \n8. BsmtQual \n\nContinuous\n1. MasVnrArea \n2. TotalBsmtSF \n3. 1stFlrSF \n4. 2ndFltSF \n5. LowQualFinSF \n6. GrLivArea\n7. BsmtFullBath\n8. BsmtHalfBath\n9. FullBath\n10. HalfBath\n11. Bedroom\n12. Kitchen\n13. TotRmsAbvGrd\n14. Fireplaces\n15. GarageCars\n16. GarageArea\n17. PoolArea\n\n'

In [326]:
# I'm assuming pre-2010 and 2010 data has to be sorted according to YrSold
# instead of YearBuilt because sorting by YearBuilt is not meaningful.
# There's only 1 house built in 2010
house.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [327]:
# Before separating into fixed_features and target_feature, I need to first
# Remove all the non-residential areas
# The following should be removed: A - Agriculture, C - Commercial, I - Industrial
# C is represented as 'C (all)' instead of C. There are no A/I in the column
house['MSZoning'].unique()

house_resi = house[(house['MSZoning'] != 'C (all)')]
house_resi.shape
# Removed 10 rows where MSZoning is 'C (all)' from the dataset

(1450, 81)

In [328]:
# Defining what are the fixed_features I'm using to predict Sale Price
# This is the first "cut"/selection I'll perform on the features
# YrSold column is left in to perform pre-2010 and 2010 filtering later
fixed_features = house_resi[['MSSubClass', 'MSZoning', 'Neighborhood', 'BldgType',
                       'HouseStyle', 'MasVnrType', 'Foundation', 'BsmtQual',
                       'MasVnrArea', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
                       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 
                       'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr',
                       'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars',
                       'GarageArea', 'PoolArea', 'YrSold']]

target_feature = house_resi[['SalePrice', 'YrSold']]

In [329]:
# Checking the quality of the data in the predictors
# There are 37 null values in BsmtQual and 8 null values in MasVnrArea
fixed_features.isnull().sum()

MSSubClass       0
MSZoning         0
Neighborhood     0
BldgType         0
HouseStyle       0
MasVnrType       8
Foundation       0
BsmtQual        37
MasVnrArea       8
TotalBsmtSF      0
1stFlrSF         0
2ndFlrSF         0
LowQualFinSF     0
GrLivArea        0
BsmtFullBath     0
BsmtHalfBath     0
FullBath         0
HalfBath         0
BedroomAbvGr     0
KitchenAbvGr     0
TotRmsAbvGrd     0
Fireplaces       0
GarageCars       0
GarageArea       0
PoolArea         0
YrSold           0
dtype: int64

In [330]:
# Checking the rows where there are null values
# BsmtQual is a categorical feature which measures the height of the basement
# 6 categories: Ex, Gd, TA, Fa, Po, NA
# Need to verify if NA is represented as NaN (Yep, NA is represented as NaN)
fixed_features[fixed_features['BsmtQual'].isnull() == True].head()
# fixed_features[fixed_features['BsmtQual'] == 'NA']

,MSSubClass,MSZoning,Neighborhood,BldgType,HouseStyle,MasVnrType,Foundation,BsmtQual,MasVnrArea,TotalBsmtSF,...,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageCars,GarageArea,PoolArea,YrSold
17,90,RL,Sawyer,Duplex,1Story,None,Slab,NaN,0.0,0,...,2,0,2,2,6,0,2,516,0,2006
39,90,RL,Edwards,Duplex,1Story,None,PConc,NaN,0.0,0,...,2,0,2,2,6,0,0,0,0,2008
90,20,RL,NAmes,1Fam,1Story,None,Slab,NaN,0.0,0,...,1,0,2,1,4,0,2,420,0,2006
102,90,RL,SawyerW,Duplex,1Story,None,Slab,NaN,0.0,0,...,2,0,4,2,8,0,2,410,0,2009
156,20,RL,NAmes,1Fam,1Story,None,CBlock,NaN,0.0,0,...,1,0,2,1,5,0,2,625,0,2006


In [331]:
# NA in BsmtQual denotes that the house has no basement
# Since we're gonna dummify categorical variables later, I think can just
# replace it with 'NA' and it will create a 1/0 dummy column for NA_BsmtQual later
fixed_features['BsmtQual'].fillna('NA', inplace=True)
fixed_features['BsmtQual'].isnull().sum()
# Checking if I have successfully filled all the NaN values

C:\Users\kengw\Anaconda3\lib\site-packages\pandas\core\generic.py:5434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


0

In [332]:
# NA is a unique value in the column
fixed_features['BsmtQual'].unique()
fixed_features[fixed_features['BsmtQual'] == 'NA'].shape
# 37 rows which are 'NA', corresponds to the 37 NaN earlier

(37, 26)

In [333]:
# Cleaning MasVnrArea & MasVnrType now
# MasVnrArea is a continuous variable, it measures the masonry veneer area
# in square feed
# MasVnrType is a categorical variable
# I'm not sure what a masonry veneer is, google search seems to suggest that
# It is some sort of foundation wall thing. 
# Given/Assuming the following:
# 1. None values are already denoted by None, therefore NaN should be a mistake 
# 2. The low number of NaN values also suggests that NaN value is likely a mistake
# I'll try to fill the NaN values with appropriate values
fixed_features[fixed_features['MasVnrArea'].isnull() == True]

,MSSubClass,MSZoning,Neighborhood,BldgType,HouseStyle,MasVnrType,Foundation,BsmtQual,MasVnrArea,TotalBsmtSF,...,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageCars,GarageArea,PoolArea,YrSold
234,60,RL,Gilbert,1Fam,2Story,NaN,PConc,Gd,NaN,860,...,2,1,4,1,8,2,2,440,0,2010
529,20,RL,Crawfor,1Fam,1Story,NaN,PConc,TA,NaN,2035,...,3,0,4,2,9,2,2,484,0,2007
650,60,FV,Somerst,1Fam,2Story,NaN,PConc,Gd,NaN,813,...,2,1,3,1,7,0,2,562,0,2008
936,20,RL,SawyerW,1Fam,1Story,NaN,PConc,Gd,NaN,1176,...,2,0,2,1,5,0,2,555,0,2009
973,20,FV,Somerst,1Fam,1Story,NaN,PConc,Gd,NaN,1428,...,2,0,3,1,6,0,2,480,0,2008
977,120,FV,Somerst,TwnhsE,1Story,NaN,PConc,Gd,NaN,1241,...,1,1,1,1,4,0,2,569,0,2007
1243,20,RL,NridgHt,1Fam,1Story,NaN,PConc,Ex,NaN,2076,...,2,1,2,1,7,1,3,850,0,2006
1278,60,RL,CollgCr,1Fam,2Story,NaN,PConc,Gd,NaN,1128,...,2,1,3,1,7,1,2,577,0,2008


In [334]:
# There are same number of NaN values for MasVnrType and MasVnrArea (8)
# These NaN rows correspond with each other
# Therefore, adopting the common sense approach, if I replace MasVnrType with 
# a None, meaning there is no measonry veneer, the corresponding area should be 0
fixed_features['MasVnrType'].value_counts()
fixed_features['MasVnrType'].fillna('None', inplace=True)
fixed_features['MasVnrArea'].fillna(0, inplace=True)

In [335]:
fixed_features['MasVnrType'].isnull().sum()
# No more NaN values in MasVnrType

0

In [336]:
fixed_features['MasVnrArea'].isnull().sum()
# No more NaN values in MasVnrArea

0

In [337]:
# Lets import all the modeling libraries
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, Lasso, LinearRegression, RidgeCV, LassoCV, LogisticRegression
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.model_selection import train_test_split

In [338]:
# Let's dummify the dataset
# Need to dummify the categorical data first
# Columns that need to be dummified are MSZoning, Neighborhood, BldgType,
# HouseStyle, MasVnrType, Foundation, BsmtQual (Checked with X_train.dtypes)

dummies = pd.get_dummies(fixed_features[['HouseStyle', 'MasVnrType', 'Foundation',
                                  'BsmtQual', 'MSZoning', 'Neighborhood',
                                  'BldgType']], drop_first=True)

# Merge the dummies back onto the original DataFrame.
fixed_features_dumb = pd.concat([fixed_features, dummies], axis=1)
# Successful creation of dummies for X_train dataset
fixed_features_dumb.shape

(1450, 76)

In [339]:
# Splitting the data into pre-2010 and 2010 first
# Naming 2010 data as house_test as it will be used as the test set
# There are 175 rows of data here
X_train = fixed_features_dumb[fixed_features_dumb['YrSold'] < 2010]
y_train = target_feature[target_feature['YrSold'] < 2010]

X_test = fixed_features_dumb[fixed_features_dumb['YrSold'] > 2009]
y_test = target_feature[target_feature['YrSold'] > 2009]

In [340]:
# Dropping the YrSold column for the train and test data
X_train.drop(labels='YrSold', axis=1, inplace=True)
y_train.drop(labels='YrSold', axis=1, inplace=True)
X_test.drop(labels='YrSold', axis=1, inplace=True)
y_test.drop(labels='YrSold', axis=1, inplace=True)

C:\Users\kengw\Anaconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [341]:
# Need to remove the original columns right now
# Removing for training set
X_train.drop(labels='HouseStyle', axis=1, inplace=True)
X_train.drop(labels='MasVnrType', axis=1, inplace=True)
X_train.drop(labels='Foundation', axis=1, inplace=True)
X_train.drop(labels='BsmtQual', axis=1, inplace=True)
X_train.drop(labels='MSZoning', axis=1, inplace=True)
X_train.drop(labels='Neighborhood', axis=1, inplace=True)
X_train.drop(labels='BldgType', axis=1, inplace=True)

# Removing for test set
X_test.drop(labels='HouseStyle', axis=1, inplace=True)
X_test.drop(labels='MasVnrType', axis=1, inplace=True)
X_test.drop(labels='Foundation', axis=1, inplace=True)
X_test.drop(labels='BsmtQual', axis=1, inplace=True)
X_test.drop(labels='MSZoning', axis=1, inplace=True)
X_test.drop(labels='Neighborhood', axis=1, inplace=True)
X_test.drop(labels='BldgType', axis=1, inplace=True)

In [342]:
# Checking through the training and test datasets
# X_train dataset is appropriately shaped (1278, 68)
# X_test dataset is appropriately shaped (172, 68)
# y_train dataset is appropriately shaped (1278, 1)
# y_test dataset is appropriately shaped (172, 1)

In [343]:
# Standardizing the data
ss = StandardScaler()
Xs_train = ss.fit_transform(X_train)
Xs_test = ss.fit_transform(X_test)

C:\Users\kengw\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\kengw\Anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\kengw\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\kengw\Anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [344]:
ys_train = ss.fit_transform(y_train)
ys_test = ss.fit_transform(y_test)

C:\Users\kengw\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\kengw\Anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\kengw\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\kengw\Anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [514]:
# Trying multiple linear regression model
lr = LinearRegression()
lr_model_1 = lr.fit(Xs_train, ys_train)


In [560]:
lr_score = cross_val_score(lr, Xs_train, ys_train, cv=10)
print('Cross-Validated Score', lr_score)
print('Mean Cross-Validated Score', np.mean(lr_score))

Cross-Validated Score [0.87218429 0.81238638 0.85725871 0.73822352 0.83462016 0.8352435
 0.86027836 0.81948425 0.585339   0.7995737 ]
Mean Cross-Validated Score 0.8014591869076462


In [562]:
lr_coef = pd.DataFrame(X_train.columns)
lr_coef['Abs Coef Value'] = pd.DataFrame(np.abs(lr.coef_.transpose()))
lr_coef.columns = [['Variable', 'Abs Coef Value']]
lr_coef.head(20)

,Variable,Abs Coef Value
0,MSSubClass,3.106415e-02
1,MasVnrArea,3.956057e-02
2,TotalBsmtSF,4.689086e-02
3,1stFlrSF,3.928513e+12
4,2ndFlrSF,4.439722e+12
5,LowQualFinSF,4.946545e+11
6,GrLivArea,5.356966e+12
7,BsmtFullBath,8.458706e-02
8,BsmtHalfBath,3.313606e-02
9,FullBath,9.143066e-02


In [365]:
yhat = cross_val_predict(lr, Xs_test, ys_test, cv=10)
yhat_df = pd.DataFrame(yhat)
ytest_df = pd.DataFrame(ys_test)


In [444]:
# The R2 scores seems good but the predicted values are somewhat close to the actual
# values, except for a few outliers
# Going to set an arbitrary cutoff point where outliers are between -5 or 5 times
# of the actual price
compare = pd.concat([yhat_df, ytest_df], axis=1)

compare.columns = ['Predicted Price', 'Actual Price'] 
compare['Predicted / Actual (Absolute)'] = compare['Predicted Price'] / compare['Actual Price']
compare['Predicted / Actual (Absolute)'] = compare['Predicted / Actual (Absolute)'].apply(lambda x: float(x))
# Made the values absolute so that it won't be skewed by negative multipliers
compare['Predicted / Actual (Absolute)'] = compare['Predicted / Actual (Absolute)'].apply(lambda x: np.abs(x))
compare['Predicted / Actual'] = compare['Predicted Price'] / compare['Actual Price']
compare.head(10)

,Predicted Price,Actual Price,Predicted / Actual (Absolute),Predicted / Actual
0,-0.580252,-0.378349,1.533643,1.533643
1,-0.652976,-0.315640,2.068737,2.068737
2,-1.312399,-0.556443,2.358553,2.358553
3,2.054468,1.590716,1.291537,1.291537
4,-0.740515,-0.171409,4.320167,4.320167
5,-0.728033,-0.880021,0.827291,0.827291
6,1.680566,1.765047,0.952137,0.952137
7,-0.583044,-0.867479,0.672113,0.672113
8,-0.278204,-0.491225,0.566348,0.566348
9,0.940591,0.010447,90.031759,90.031759


In [449]:
# Filtering out the outliers
# Noticed that very few rows are removed as outliers, only 14
compare_no_outlier_abs = compare[(compare['Predicted / Actual (Absolute)'] < 6)]
compare_no_outlier = compare[(compare['Predicted / Actual'] < 6) & (compare['Predicted / Actual'] > -6)]


In [451]:
# On average, the absolute (predicted/actual) is about 1.29x
# Pretty close I would say
print(compare_no_outlier_abs['Predicted / Actual (Absolute)'].mean())
print(compare_no_outlier['Predicted / Actual'].mean())

1.2917710821240855
1.1654428108220367


In [466]:
# Lets try now with Lasso
# Finding the optimal alpha value for lasso
optimal_lasso = LassoCV(n_alphas=500, cv=10, verbose=1)
optimal_lasso.fit(Xs_train, ys_train)

C:\Users\kengw\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
..............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

LassoCV(alphas=None, copy_X=True, cv=10, eps=0.001, fit_intercept=True,
    max_iter=1000, n_alphas=500, n_jobs=None, normalize=False,
    positive=False, precompute='auto', random_state=None,
    selection='cyclic', tol=0.0001, verbose=1)

In [471]:
# This is the optimal alpha value for lasso
print(optimal_lasso.alpha_)

# Creating a model with the optimal lasso
lasso = Lasso(alpha=optimal_lasso.alpha_)
lasso_scores = cross_val_score(lasso, Xs_train, ys_train, cv=10)
lasso.fit(Xs_train, ys_train)

# This performs slightly better than the mlr model
print(lasso_scores)
print(np.mean(lasso_scores))

0.004918645014038293
[0.87682522 0.8231686  0.86453425 0.75336467 0.82497601 0.84371441
 0.87652592 0.81214196 0.56170401 0.81398036]
0.8050935397833839


In [474]:
# Displaying the lasso coefficients
lasso_coefs = pd.DataFrame({'Variable': X_train.columns, 'Coefficient':lasso.coef_, 'Absolute Coefficient':np.abs(lasso.coef_)})
lasso_coefs.sort_values('Absolute Coefficient', inplace=True, ascending=False)

In [488]:
# These features are deemed to be insignificant in determine the Sale Price
# By the lasso function
# If given more time, I would like to try to manually remove these features to
# Fit a new model and see if the score improves
lasso_coefs[lasso_coefs['Absolute Coefficient'] == 0].head(3)

,Variable,Coefficient,Absolute Coefficient
4,2ndFlrSF,0.0,0.0
20,HouseStyle_2.5Fin,0.0,0.0
40,Neighborhood_Blueste,0.0,0.0


In [487]:
# GrLivArea, BsmtQual_TA, BsmtQual_Gd, Neighborhood_NridgHt, Neighborghood_NoRidge
# GarageCars, Neighborhood_stoneBr, BsmtQual_Fa have coef values > 0.1
# They seem to be strong predictors for Sale Price or amongt the best estimates
# For sale price

# GrLivArea makes sense as its a continuous variable that measures non-basement
# living area in square feet (i.e. size of house)

# BsmtQual makes sense also as the height of the basement is likely to determine
# How valued the house is

# GarageCars makes sense also, this measures the number of cars that can fit
# into the garage. Pricer houses likely to have garage that can fit more cars, vice versa
lasso_coefs[lasso_coefs['Absolute Coefficient'] != 0].head(8)

,Variable,Coefficient,Absolute Coefficient
6,GrLivArea,0.370492,0.370492
36,BsmtQual_TA,-0.266785,0.266785
34,BsmtQual_Gd,-0.237610,0.237610
55,Neighborhood_NridgHt,0.157257,0.157257
54,Neighborhood_NoRidge,0.133379,0.133379
15,GarageCars,0.118208,0.118208
61,Neighborhood_StoneBr,0.108852,0.108852
33,BsmtQual_Fa,-0.106872,0.106872


In [489]:
# Making cross-validated predictions with the best lasso model
yhat_lasso = cross_val_predict(lasso, Xs_test, ys_test, cv=10)
yhat_lasso_df = pd.DataFrame(yhat_lasso)
ytest_lasso_df = pd.DataFrame(ys_test)

In [490]:
# Comparison table of predicted price against actual price
compare1 = pd.concat([yhat_lasso_df, ytest_lasso_df], axis=1)

compare1.columns = ['Predicted Price', 'Actual Price'] 
compare1['Predicted / Actual (Absolute)'] = compare1['Predicted Price'] / compare1['Actual Price']
compare1['Predicted / Actual (Absolute)'] = compare1['Predicted / Actual (Absolute)'].apply(lambda x: float(x))
# Made the values absolute so that it won't be skewed by negative multipliers
compare1['Predicted / Actual (Absolute)'] = compare1['Predicted / Actual (Absolute)'].apply(lambda x: np.abs(x))
compare1['Predicted / Actual'] = compare1['Predicted Price'] / compare1['Actual Price']
compare1.head(10)

,Predicted Price,Actual Price,Predicted / Actual (Absolute),Predicted / Actual
0,-0.539698,-0.378349,1.426457,1.426457
1,-0.646162,-0.315640,2.047151,2.047151
2,-1.141480,-0.556443,2.051388,2.051388
3,1.922705,1.590716,1.208704,1.208704
4,-0.481326,-0.171409,2.808055,2.808055
5,-0.769365,-0.880021,0.874257,0.874257
6,1.698236,1.765047,0.962148,0.962148
7,-0.582403,-0.867479,0.671374,0.671374
8,-0.249400,-0.491225,0.507710,0.507710
9,1.090959,0.010447,104.424655,104.424655


In [492]:
# Lasso model performs slightly better than a linear regression model
compare1_no_outlier_abs = compare1[(compare1['Predicted / Actual (Absolute)'] < 6)]
compare1_no_outlier = compare1[(compare1['Predicted / Actual'] < 6) & (compare1['Predicted / Actual'] > -6)]
print(compare1_no_outlier_abs['Predicted / Actual (Absolute)'].mean())
print(compare1_no_outlier['Predicted / Actual'].mean())

1.2164289642626647
1.0163849053374026


In [ ]:
# If there's time, attempt Ridge & Elastic Net
# Even more time, carry out gridsearch to find the best parameters for Ridge/Lasso

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 2. Determine any value of *changeable* property characteristics unexplained by the *fixed* ones.

---

Now that you have a model that estimates the price of a house based on its static characteristics, we can move forward with part 2 and 3 of the plan: what are the costs/benefits of quality, condition, and renovations?

There are two specific requirements for these estimates:
1. The estimates of effects must be in terms of dollars added or subtracted from the house value. 
2. The effects must be on the variance in price remaining from the first model.

The residuals from the first model (training and testing) represent the variance in price unexplained by the fixed characteristics. Of that variance in price remaining, how much of it can be explained by the easy-to-change aspects of the property?

---

**Your goals:**
1. Evaluate the effect in dollars of the renovate-able features. 
- How would your company use this second model and its coefficients to determine whether they should buy a property or not? Explain how the company can use the two models you have built to determine if they can make money. 
- Investigate how much of the variance in price remaining is explained by these features.
- Do you trust your model? Should it be used to evaluate which properties to buy and fix up?

In [570]:
# Identify renovatable features:
# Some examples of things that ARE renovate-able:
# Roof and exterior features, "Quality" metrics, such as kitchen quality
# "Condition" metrics, such as condition of garage, Heating and electrical components

# Renovatable features:
# LotFrontage, LotArea, Street, Alley, LotShape, LandContour, OverallQual,
# OverallCond, YearBuilt, RoofStyle, RoofMatl, Exterior1st, Exterior2nd, 
# ExterQual, ExterCond, BmstCond, BmstExposure, Heating, HeatingQC, Electrical
# KitchenQual, Functional, FireplaceQu, GarageFinish, GarageQual, GarageCond,
# PoolQC, YrSold


# y_test is still ys_test
# y_train is still ys_train

predictors1 = house_resi[['LotArea', 'Street', 
                          'LotShape', 'LandContour', 'OverallQual', 'OverallCond',
                          'YearBuilt', 'RoofStyle', 'RoofMatl', 'Exterior1st', 
                          'Exterior2nd', 'ExterQual', 'ExterCond', 'BsmtCond',
                          'BsmtExposure', 'Heating', 'HeatingQC', 'Electrical',
                          'KitchenQual', 'Functional', 
                          
                          'YrSold']]



In [571]:
predictors1.info()
# In interest of time, gonna drop LotFrontage, Alley, FireplaceQu, GarageFinish
# GarageQual, GarageCond, PoolQc from the predictors as they have too many null
# values to cleanup

# Remaining problematic ones are:
# 1. BsmtCond 2. BsmtExposure 3. Electrical
# Lets investigate further

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1450 entries, 0 to 1459
Data columns (total 21 columns):
LotArea         1450 non-null int64
Street          1450 non-null object
LotShape        1450 non-null object
LandContour     1450 non-null object
OverallQual     1450 non-null int64
OverallCond     1450 non-null int64
YearBuilt       1450 non-null int64
RoofStyle       1450 non-null object
RoofMatl        1450 non-null object
Exterior1st     1450 non-null object
Exterior2nd     1450 non-null object
ExterQual       1450 non-null object
ExterCond       1450 non-null object
BsmtCond        1413 non-null object
BsmtExposure    1412 non-null object
Heating         1450 non-null object
HeatingQC       1450 non-null object
Electrical      1449 non-null object
KitchenQual     1450 non-null object
Functional      1450 non-null object
YrSold          1450 non-null int64
dtypes: int64(5), object(16)
memory usage: 249.2+ KB


In [575]:
# I'm assuming that the null values are denoted as NaN instead of NA for no basement
predictors1['BsmtCond'].unique()
predictors1['BsmtCond'].fillna('NA', inplace=True)

In [578]:
# I'm assuming that the null values are denoted as NaN instead of NA for no basement
predictors1['BsmtExposure'].unique()
predictors1['BsmtExposure'].fillna('NA', inplace=True)

In [586]:
# Think this is probably a mistake that there is a NaN value
# Gonna ill this up with the most common value which is SBrkr
predictors1[predictors1['Electrical'].isnull()]
predictors1['Electrical'].fillna('SBrkr', inplace=True)

In [591]:
# Checking if all nulls are filled in (They are)
predictors1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1450 entries, 0 to 1459
Data columns (total 21 columns):
LotArea         1450 non-null int64
Street          1450 non-null object
LotShape        1450 non-null object
LandContour     1450 non-null object
OverallQual     1450 non-null int64
OverallCond     1450 non-null int64
YearBuilt       1450 non-null int64
RoofStyle       1450 non-null object
RoofMatl        1450 non-null object
Exterior1st     1450 non-null object
Exterior2nd     1450 non-null object
ExterQual       1450 non-null object
ExterCond       1450 non-null object
BsmtCond        1450 non-null object
BsmtExposure    1450 non-null object
Heating         1450 non-null object
HeatingQC       1450 non-null object
Electrical      1450 non-null object
KitchenQual     1450 non-null object
Functional      1450 non-null object
YrSold          1450 non-null int64
dtypes: int64(5), object(16)
memory usage: 249.2+ KB


In [594]:
# Creating dummies for the categorical data
dummy = pd.get_dummies(predictors1[['Street', 'LotShape', 'LandContour',
                                  'RoofStyle', 'RoofMatl', 'Exterior1st',
                                  'Exterior2nd', 'ExterQual', 'ExterCond',
                                  'BsmtCond', 'BsmtExposure', 'Heating',
                                  'HeatingQC', 'Electrical', 'KitchenQual',
                                  'Functional']], drop_first=True)
dummy.shape

(1450, 85)

In [596]:
# Merge the dummies back onto the original DataFrame.
predictors1_dumb = pd.concat([predictors1, dummy], axis=1)
# Successful creation of dummies for X_train dataset
predictors1_dumb.shape

(1450, 106)

In [598]:
# Removing the original columns
predictors1_dumb.drop(labels='Street', axis=1, inplace=True)
predictors1_dumb.drop(labels='LotShape', axis=1, inplace=True)
predictors1_dumb.drop(labels='LandContour', axis=1, inplace=True)
predictors1_dumb.drop(labels='RoofStyle', axis=1, inplace=True)
predictors1_dumb.drop(labels='RoofMatl', axis=1, inplace=True)
predictors1_dumb.drop(labels='Exterior1st', axis=1, inplace=True)
predictors1_dumb.drop(labels='Exterior2nd', axis=1, inplace=True)
predictors1_dumb.drop(labels='ExterQual', axis=1, inplace=True)
predictors1_dumb.drop(labels='ExterCond', axis=1, inplace=True)
predictors1_dumb.drop(labels='BsmtCond', axis=1, inplace=True)
predictors1_dumb.drop(labels='BsmtExposure', axis=1, inplace=True)
predictors1_dumb.drop(labels='Heating', axis=1, inplace=True)
predictors1_dumb.drop(labels='HeatingQC', axis=1, inplace=True)
predictors1_dumb.drop(labels='Electrical', axis=1, inplace=True)
predictors1_dumb.drop(labels='KitchenQual', axis=1, inplace=True)
predictors1_dumb.drop(labels='Functional', axis=1, inplace=True)

In [599]:
predictors1_dumb.shape

(1450, 90)

In [603]:
# y_test is still ys_test
# y_train is still ys_train

X_train_q2 = predictors1_dumb[predictors1_dumb['YrSold'] < 2010]
ys_train_q2 = ys_train

X_test_q2 = predictors1_dumb[predictors1_dumb['YrSold'] > 2009]
ys_test_q2 = ys_test

In [604]:
# Removing the YrSold column
X_train_q2.drop(labels='YrSold', axis=1, inplace=True)
X_test_q2.drop(labels='YrSold', axis=1, inplace=True)

In [605]:
# Standardizing the data
Xs_train_q2 = ss.fit_transform(X_train_q2)
Xs_test_q2 = ss.fit_transform(X_test_q2)

C:\Users\kengw\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\kengw\Anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\kengw\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\kengw\Anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [608]:
# Trying with lasso regression
optimal_lasso_q2 = LassoCV(n_alphas=500, cv=10, verbose=1)
optimal_lasso_q2.fit(Xs_train_q2, ys_train_q2)

C:\Users\kengw\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
..............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

LassoCV(alphas=None, copy_X=True, cv=10, eps=0.001, fit_intercept=True,
    max_iter=1000, n_alphas=500, n_jobs=None, normalize=False,
    positive=False, precompute='auto', random_state=None,
    selection='cyclic', tol=0.0001, verbose=1)

In [610]:
# This is the optimal alpha value for lasso
print(optimal_lasso_q2.alpha_)

# Creating a model with the optimal lasso
lasso_q2 = Lasso(alpha=optimal_lasso_q2.alpha_)
lasso_scores_q2 = cross_val_score(lasso_q2, Xs_train_q2, ys_train_q2, cv=10)
lasso_q2.fit(Xs_train_q2, ys_train_q2)

# This performs slightly better than the mlr model
print(lasso_scores_q2)
print(np.mean(lasso_scores_q2))

0.008313836892677978
[0.77234135 0.76227112 0.75948547 0.65940548 0.72848238 0.69299018
 0.65874744 0.68581477 0.58453791 0.75125636]
0.7055332461273751


In [615]:
# Displaying the lasso coefficients
lasso_coefs_q2 = pd.DataFrame({'Variable': X_train_q2.columns, 'Coefficient':lasso_q2.coef_, 'Absolute Coefficient':np.abs(lasso_q2.coef_)})
lasso_coefs_q2.sort_values('Absolute Coefficient', inplace=True, ascending=False)
lasso_coefs_q2[lasso_coefs_q2['Absolute Coefficient'] != 0]

# OverallQual, KitchenQual, ExterQual, LotArea are the strongest renovatable
# Predictors for Sale Price

,Variable,Coefficient,Absolute Coefficient
1,OverallQual,0.525533,0.525533
82,KitchenQual_TA,-0.266676,0.266676
81,KitchenQual_Gd,-0.211105,0.211105
54,ExterQual_TA,-0.161349,0.161349
0,LotArea,0.140385,0.140385
22,RoofMatl_WdShngl,0.090558,0.090558
53,ExterQual_Gd,-0.083015,0.083015
63,BsmtExposure_Gd,0.079229,0.079229
80,KitchenQual_Fa,-0.068246,0.068246
11,RoofStyle_Gable,-0.067129,0.067129


In [616]:
# Making cross-validated predictions with the best lasso model
yhat_lasso_q2 = cross_val_predict(lasso_q2, Xs_test_q2, ys_test_q2, cv=10)
yhat_lasso_df_q2 = pd.DataFrame(yhat_lasso_q2)
ytest_lasso_df_q2 = pd.DataFrame(ys_test_q2)

In [617]:
compareq2 = pd.concat([yhat_lasso_df_q2, ytest_lasso_df_q2], axis=1)

compareq2.columns = ['Predicted Price', 'Actual Price'] 
compareq2['Predicted / Actual (Absolute)'] = compareq2['Predicted Price'] / compareq2['Actual Price']
compareq2['Predicted / Actual (Absolute)'] = compareq2['Predicted / Actual (Absolute)'].apply(lambda x: float(x))
# Made the values absolute so that it won't be skewed by negative multipliers
compareq2['Predicted / Actual (Absolute)'] = compareq2['Predicted / Actual (Absolute)'].apply(lambda x: np.abs(x))
compareq2['Predicted / Actual'] = compareq2['Predicted Price'] / compareq2['Actual Price']

In [618]:
compareq2.head(20)

,Predicted Price,Actual Price,Predicted / Actual (Absolute),Predicted / Actual
0,0.000124,-0.378349,0.000329,-0.000329
1,-0.625203,-0.315640,1.980749,1.980749
2,-0.546907,-0.556443,0.982863,0.982863
3,0.987817,1.590716,0.620989,0.620989
4,-0.349058,-0.171409,2.036405,2.036405
5,-0.564109,-0.880021,0.641018,0.641018
6,2.857149,1.765047,1.618738,1.618738
7,-2.222074,-0.867479,2.561530,2.561530
8,-0.853086,-0.491225,1.736650,1.736650
9,0.745280,0.010447,71.336856,71.336856


In [622]:
print(np.mean(compareq2['Predicted / Actual (Absolute)']))
print(np.mean(compareq2['Predicted / Actual']))

2.198120900084295
1.483715758058879


<img src="http://imgur.com/GCAf1UX.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 3. What property characteristics predict an "abnormal" sale?

---

The `SaleCondition` feature indicates the circumstances of the house sale. From the data file, we can see that the possibilities are:

       Normal	Normal Sale
       Abnorml	Abnormal Sale -  trade, foreclosure, short sale
       AdjLand	Adjoining Land Purchase
       Alloca	Allocation - two linked properties with separate deeds, typically condo with a garage unit	
       Family	Sale between family members
       Partial	Home was not completed when last assessed (associated with New Homes)
       
One of the executives at your company has an "in" with higher-ups at the major regional bank. His friends at the bank have made him a proposal: if he can reliably indicate what features, if any, predict "abnormal" sales (foreclosures, short sales, etc.), then in return the bank will give him first dibs on the pre-auction purchase of those properties (at a dirt-cheap price).

He has tasked you with determining (and adequately validating) which features of a property predict this type of sale. 

---

**Your task:**
1. Determine which features predict the `Abnorml` category in the `SaleCondition` feature.
- Justify your results.

This is a challenging task that tests your ability to perform classification analysis in the face of severe class imbalance. You may find that simply running a classifier on the full dataset to predict the category ends up useless: when there is bad class imbalance classifiers often tend to simply guess the majority class.

It is up to you to determine how you will tackle this problem. I recommend doing some research to find out how others have dealt with the problem in the past. Make sure to justify your solution. Don't worry about it being "the best" solution, but be rigorous.

Be sure to indicate which features are predictive (if any) and whether they are positive or negative predictors of abnormal sales.

In [26]:
# A: